In [8]:
import ast
import json
import chess
import numpy as np

from NeuralNet import NeuralNet
from Game import Game

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def convert_to_pis(evaluations):
    result = np.zeros(64*64)
    best, worst = evaluations[0][1], evaluations[-1][1]
    if best == worst:
        result[evaluations[0][0]] = 1
    else:
        normalized = [(evaluation - worst)/(best - worst) for _, evaluation in evaluations]
        normalized = [float(i)/sum(normalized) for i in normalized]
        for i, value in enumerate(normalized):
            result[evaluations[i][0]] = value

    return result

In [18]:
data_file = "../../chess_data/full_data_unique.json"
game = Game()

# data_file = "../../chess_data/sample.json"

with open(data_file) as file:
    lines = file.readlines()

input_boards = []
target_pis = []
target_vs = []

for line in lines:
    parsed_line = ast.literal_eval(line)
    fen = parsed_line[0]
    cannonical_fen = game.getCanonicalForm(fen, chess.WHITE if fen.split(' ')[1] == 'w' else chess.BLACK)
    input_boards.append(cannonical_fen)
    target_vs.append(parsed_line[1])
    target_pis.append(convert_to_pis(parsed_line[2]))

examples = zip(input_boards, target_pis, target_vs)

In [5]:
nn = NeuralNet()

2022-08-25 22:34:50.876649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 22:34:50.952632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 22:34:50.952816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 22:34:50.953856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [19]:
input_boards, target_pis, target_vs = list(zip(*examples))
input_boards = np.asarray(input_boards)
input_boards = np.asarray([nn._from_fen_to_array(board) for board in input_boards]).reshape(-1, 12, 8, 8)
target_pis = np.asarray(target_pis)
target_vs = np.asarray(target_vs)
nn.model.fit(x = input_boards, y = [target_pis, target_vs], batch_size = 64, epochs = 10, validation_split=0.2)

2022-08-25 22:55:11.292999: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1111965696 exceeds 10% of free system memory.
2022-08-25 22:55:12.646769: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1111965696 exceeds 10% of free system memory.


Epoch 1/10


2022-08-25 22:55:13.272445: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-25 22:55:13.273230: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-25 22:55:13.599045: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.43GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-25 22:55:13.599130: W tensorflow/core/common_runtime/bfc_allocato

  25/1061 [..............................] - ETA: 4:22 - loss: 206275.7812 - pi_loss: 7.0589 - v_loss: 206268.7188